In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

# Example number of species in your dataset
num_species = 3  # Replace with the real number of species

# Input layer
inputs = Input(shape=(2,), name='env_inputs')  # Temp and Phytoplankton

# Shared hidden layers
x = Dense(32, activation='relu')(inputs)
x = Dense(32, activation='relu')(x)

# Output 1: Hunting area (binary)
hunting_output = Dense(1, activation='sigmoid', name='hunting_area')(x)

# Output 2: Species probabilities (multi-output regression/probabilities)
species_output = Dense(num_species, activation='sigmoid', name='species_probs')(x)

# Model
model = Model(inputs=inputs, outputs=[hunting_output, species_output])

# Compile with multi-task losses
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={
        'hunting_area': 'binary_crossentropy',
        'species_probs': 'mse'  # Could also try 'mae'
    },
    metrics={
        'hunting_area': ['accuracy'],
        'species_probs': ['mae']
    }
)

model.summary()


In [ ]:
import pandas as pd
import numpy as np

# Inputs
X = df[['Temp', 'Phytoplankton']].values

# Outputs
y_hunting = df['Hunting_area'].values
y_species = df[['Great_White', 'Tiger', 'Bull']].values  # Replace with your species columns

# Train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_h_train, y_h_test, y_s_train, y_s_test = train_test_split(
    X, y_hunting, y_species, test_size=0.2, random_state=42
)

# Train the model
history = model.fit(
    X_train,
    {'hunting_area': y_h_train, 'species_probs': y_s_train},
    validation_data=(X_test, {'hunting_area': y_h_test, 'species_probs': y_s_test}),
    epochs=50,
    batch_size=32
)


In [ ]:
# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save TFLite file
with open("shark_model.tflite", "wb") as f:
    f.write(tflite_model)
